In [1]:
%pip install pyannote.audio
%pip install numpy==1.26

  Using cached safetensors-0.6.2-cp38-abi3-macosx_11_0_arm64.whl.metadata (4.1 kB)
  Using cached torch-2.9.1-cp312-none-macosx_11_0_arm64.whl.metadata (30 kB)
  Using cached pyparsing-3.2.5-py3-none-any.whl.metadata (5.0 kB)
  Using cached scikit_learn-1.7.2-cp312-cp312-macosx_12_0_arm64.whl.metadata (11 kB)
  Using cached scipy-1.16.3-cp312-cp312-macosx_14_0_arm64.whl.metadata (62 kB)
  Using cached setuptools-80.9.0-py3-none-any.whl.metadata (6.6 kB)
  Using cached networkx-3.5-py3-none-any.whl.metadata (6.3 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached joblib-1.5.2-py3-none-any.whl.metadata (5.6 kB)
  Using cached threadpoolctl-3.6.0-py3-none-any.whl.metadata (13 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 891.5/891.5 kB 18.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 827.9/827.9 kB 16.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.1/8.

In [2]:
import os
from dotenv import load_dotenv

load_dotenv()

HUGGING_FACE_TOKEN = os.getenv("HUGGING_FACE_TOKEN")

In [3]:
# instantiate the pipeline
from pyannote.audio import Pipeline

pipeline = Pipeline.from_pretrained(
  "pyannote/speaker-diarization-3.1",
  use_auth_token=HUGGING_FACE_TOKEN
)

Matplotlib is building the font cache; this may take a moment.
/Users/solbi/Documents/GitHub/study-gpt_agent/venv/lib/python3.12/site-packages/pyannote/audio/core/io.py:47: UserWarning: 
torchcodec is not installed correctly so built-in audio decoding will fail. Solutions are:
* use audio preloaded in-memory as a {'waveform': (channel, time) torch.Tensor, 'sample_rate': int} dictionary;
* fix torchcodec installation. Error message was:

Could not load libtorchcodec. Likely causes:
          1. FFmpeg is not properly installed in your environment. We support
             versions 4, 5, 6, and 7 on all platforms, and 8 on Mac and Linux.
          2. The PyTorch version (2.9.1) is not compatible with
             this version of TorchCodec. Refer to the version compatibility
             table:
             https://github.com/pytorch/torchcodec?tab=readme-ov-file#installing-torchcodec.
          3. Another runtime dependency; see exceptions below.
        The following exceptions were rai

TypeError: Pipeline.from_pretrained() got an unexpected keyword argument 'use_auth_token'

In [4]:
import torch

# cuda가 사용 가능한 경우 cuda를 사용하도록 설정
if torch.cuda.is_available():
    pipeline.to(torch.device("cuda"))
    print('cuda is available')
else:
    print('cuda is not available')

cuda is not available


In [7]:
# run the pipeline on an audio file
# diarization = pipeline("audio.wav")
diarization = pipeline("../audio/싼기타_비싼기타.mp3")

# dump the diarization output to disk using RTTM format
with open("싼기타_비싼기타.rttm", "w", encoding='utf-8') as rttm:
    diarization.write_rttm(rttm)


NameError: name 'pipeline' is not defined

In [8]:
# RTTM을 CSV로 변환
import pandas as pd
rttm_path = "./싼기타_비싼기타.rttm"

df_rttm = pd.read_csv(
    rttm_path,      # rttm 파일 경로
    sep=' ',        # 구분자는 띄어쓰기
    header=None,    # 헤더는 없음
    names=['type', 'file', 'chnl', 'start', 'duration', 'C1', 'C2', 'speaker_id', 'C3', 'C4'] 
)

display(df_rttm)

FileNotFoundError: [Errno 2] No such file or directory: './싼기타_비싼기타.rttm'

In [9]:
# start + duration을 end로 변환
df_rttm['end'] = df_rttm['start'] + df_rttm['duration']

display(df_rttm)

NameError: name 'df_rttm' is not defined

In [10]:
df_rttm["number"] = None	# number 열 만들고 None으로 초기화
df_rttm.at[0, "number"] = 0

display(df_rttm)

NameError: name 'df_rttm' is not defined

In [11]:
for i in range(1, len(df_rttm)):
    if df_rttm.at[i, "speaker_id"] != df_rttm.at[i-1, "speaker_id"]:
        df_rttm.at[i, "number"] = df_rttm.at[i-1, "number"] + 1
    else:
        df_rttm.at[i, "number"] = df_rttm.at[i-1, "number"]

display(df_rttm.head(10)) 

NameError: name 'df_rttm' is not defined

In [ ]:
df_rttm_grouped = df_rttm.groupby("number").agg(
    start=pd.NamedAgg(column='start', aggfunc='min'),
    end=pd.NamedAgg(column='end', aggfunc='max'),
    speaker_id=pd.NamedAgg(column='speaker_id', aggfunc='first')
)

display(df_rttm_grouped)

In [ ]:
df_rttm_grouped["duration"] = df_rttm_grouped["end"] - df_rttm_grouped["start"]
df_rttm_grouped = df_rttm_grouped.reset_index(drop=True)
display(df_rttm_grouped)

In [ ]:
df_rttm_grouped.to_csv(
    "싼기타_비싼기타_rttm.csv",
    sep=',',
    index=False
)